In [5]:
!/opt/homebrew/Cellar/jupyterlab/4.4.2_1/libexec/bin/python -m pip install pulp
!/opt/homebrew/Cellar/jupyterlab/4.4.2_1/libexec/bin/python -m pip install tabulate


The system cannot find the path specified.
The system cannot find the path specified.


In [6]:
import pandas as pd
import pulp
from collections import defaultdict
import math
from tabulate import tabulate


capacities_df = pd.read_csv('../Data/output/final/metro_capacities_updated.csv')
schedule_df = pd.read_csv('../Data/output/final/metro_schedule.csv')

print("Columns in metro_capacities_updated.csv:", capacities_df.columns.tolist())
print("Columns in metro_schedule.csv:", schedule_df.columns.tolist())

print("\nUnique stations in capacities_df:", capacities_df['Station'].unique())

transfer_stations = {
    'Yenikapı', 'Levent', 'Şişli-Mecidiyeköy', 'Kirazlı', 'Mahmutbey',
    'Kozyatağı', 'Dudullu', 'Yenibosna', 'İkitelli Sanayi',
    'Bostancı', 'Kağıthane', 'Gayrettepe'
}

schedule_df = schedule_df[
    (schedule_df['Day_Type'] == 'Weekday') & 
    (schedule_df['Peak_Hour_Indicator'] == 'Yes')
]

print("\nFiltered schedule data shape:", schedule_df.shape)
print("Unique lines in filtered schedule:", schedule_df['Line'].unique())

station_lines = capacities_df.groupby('Station')['Line'].nunique().to_dict()
weights = {station: station_lines.get(station, 1) for station in transfer_stations}

print("\nStation weights:", weights)

unique_dates = capacities_df['transition_date'].unique()

all_results = []
all_weighted_results = []

for date in unique_dates:
    daily_capacities = capacities_df[
        (capacities_df['transition_date'] == date) &
        (capacities_df['Station'].isin(transfer_stations)) &
        (capacities_df['transition_hour'].isin(schedule_df['transition_hour']))
    ]
    
    print(f"\nDate {date}:")
    print("Daily capacities shape:", daily_capacities.shape)
    print("Unique stations in daily capacities:", daily_capacities['Station'].unique())
    
    print(f"\nAdjusted RemainingCapacity for date {date}:")
    for station in transfer_stations:
        station_data = daily_capacities[daily_capacities['Station'] == station]
        if not station_data.empty:
            capacities = {}
            for _, row in station_data.iterrows():
                key = (row['Line'], row['Direction'], row['transition_hour'])
                capacities[key] = row['RemainingCapacity'] + 100
            print(f"  {station}: {capacities}")
        else:
            print(f"  {station}: No data found")
    
    demand_data = daily_capacities.groupby(['Station', 'Line', 'Direction', 'transition_hour'])['number_of_passenger'].sum().reset_index()
    
    trips_data = schedule_df.groupby(['Line', 'Direction', 'transition_hour'])['Number_of_Trips'].sum().reset_index()
    
    di = {}
    ci = {}
    ti = {}
    
    for index, row in demand_data.iterrows():
        key = (row['Station'], row['Line'], row['Direction'], row['transition_hour'], date)
        di[key] = row['number_of_passenger']
    
    for _, row in daily_capacities.iterrows():
        key = (row['Station'], row['Line'], row['Direction'], row['transition_hour'], date)
        ci[key] = row['RemainingCapacity'] + 100
    
    for index, row in trips_data.iterrows():
        line, direction, hour = row['Line'], row['Direction'], row['transition_hour']
        matching_stations = daily_capacities[
            (daily_capacities['Line'] == line) & 
            (daily_capacities['Direction'] == direction) &
            (daily_capacities['Station'].isin(transfer_stations))
        ]['Station'].unique()
        for station in matching_stations:
            key = (station, line, direction, hour, date)
            if key in di:
                ti[key] = row['Number_of_Trips']
    
    prob = pulp.LpProblem(f"Metro_Optimization_{date}", pulp.LpMinimize)
    
    xi = pulp.LpVariable.dicts(
        "Additional_Trips",
        [(s, l, d, h, date) for s, l, d, h, _ in di.keys()],
        lowBound=0,
        cat='Integer'
    )
    
    prob += pulp.lpSum(xi[s, l, d, h, date] for s, l, d, h, _ in di.keys()), f"Total_Additional_Trips_{date}"
    
    # Constraints
    for key in di:
        s, l, d, h, _ = key
        if key in ci:
            prob += (
                ti.get(key, 0) * ci[key] + xi[s, l, d, h, date] * ci[key] >= di[key],
                f"Demand_Constraint_{s}_{l}_{d}_{h}_{date}"
            )
    
    prob.solve()
    
    for key in xi:
        if xi[key].varValue is not None and xi[key].varValue > 0:
            s, l, d, h, _ = key
            all_results.append({
                'Date': date,
                'Station': s,
                'Line': l,
                'Direction': d,
                'Hour': h,
                'Additional_Trips': int(xi[key].varValue),
                'Passenger_Demand': di[key],
                'Current_Trips': ti.get(key, 0),
                'Capacity_Per_Trip': ci.get(key, 100)
            })
    
    prob_weighted = pulp.LpProblem(f"Metro_Optimization_Weighted_{date}", pulp.LpMinimize)
    
    xi_weighted = pulp.LpVariable.dicts(
        "Additional_Trips_Weighted",
        [(s, l, d, h, date) for s, l, d, h, _ in di.keys()],
        lowBound=0,
        cat='Integer'
    )
    
    prob_weighted += pulp.lpSum(
        weights[s] * xi_weighted[s, l, d, h, date] for s, l, d, h, _ in di.keys()
    ), f"Total_Weighted_Additional_Trips_{date}"
    
    for key in di:
        s, l, d, h, _ = key
        if key in ci:
            prob_weighted += (
                ti.get(key, 0) * ci[key] + xi_weighted[s, l, d, h, date] * ci[key] >= di[key],
                f"Demand_Constraint_Weighted_{s}_{l}_{d}_{h}_{date}"
            )
    
    prob_weighted.solve()
    
    for key in xi_weighted:
        if xi_weighted[key].varValue is not None and xi_weighted[key].varValue > 0:
            s, l, d, h, _ = key
            all_weighted_results.append({
                'Date': date,
                'Station': s,
                'Line': l,
                'Direction': d,
                'Hour': h,
                'Additional_Trips': int(xi_weighted[key].varValue),
                'Weight': weights[s],
                'Passenger_Demand': di[key],
                'Current_Trips': ti.get(key, 0),
                'Capacity_Per_Trip': ci.get(key, 100)
            })

results_df = pd.DataFrame(all_results)
weighted_results_df = pd.DataFrame(all_weighted_results)

avg_results = results_df.groupby(['Station', 'Line', 'Direction', 'Hour']).agg({
    'Additional_Trips': lambda x: math.ceil(x.mean()),
    'Passenger_Demand': 'mean',
    'Current_Trips': 'mean',
    'Capacity_Per_Trip': 'mean'
}).reset_index()

avg_weighted_results = weighted_results_df.groupby(['Station', 'Line', 'Direction', 'Hour']).agg({
    'Additional_Trips': lambda x: math.ceil(x.mean()),
    'Passenger_Demand': 'mean',
    'Current_Trips': 'mean',
    'Capacity_Per_Trip': 'mean',
    'Weight': 'mean'
}).reset_index()

print("\nUnweighted Optimization (Average Additional Trips):")
print(tabulate(avg_results, headers='keys', tablefmt='grid', showindex=False))

print("\nWeighted Optimization (Average Additional Trips):")
print(tabulate(avg_weighted_results, headers='keys', tablefmt='grid', showindex=False))


results_df.to_csv('../Data/output/results/optimization_results.csv', index=False)
weighted_results_df.to_csv('../Data/output/results/weighted_optimization_results.csv', index=False)
avg_results.to_csv('../Data/output/results/avg_optimization_results.csv', index=False)
avg_weighted_results.to_csv('../Data/output/results/avg_weighted_optimization_results.csv', index=False)

report_lines = [
    "Istanbul Metro Optimization Report",
    "",
    "Overview",
    "--------",
    "This report analyzes overcrowding at key transfer stations in Istanbul's metro system during peak hours on a typical weekday. The goal is to determine the minimum number of additional trips needed to meet passenger demand, using adjusted remaining capacity data (RemainingCapacity + 40) to model overcrowding more realistically and reduce the number of additional trips. An Integer Linear Programming (ILP) model optimizes additional trips for each day, with results averaged (and rounded up) across days to estimate typical needs. Unweighted (minimizing total trips) and weighted (prioritizing stations with more lines) optimizations are performed.",
    "",
    f"Transfer stations: {', '.join(transfer_stations)}.",
    "",
    "Methodology",
    "-----------",
    "- Data Sources:",
    "  - metro_capacities_updated.csv: Actual passenger demand and remaining capacity per station, line, direction, hour, and date.",
    "  - metro_schedule.csv: Number of trips per line, direction, and hour during peak hours (Weekday, Peak_Hour_Indicator = 'Yes').",
    "- Key Metrics:",
    "  - Demand (di): Actual number of passengers.",
    "  - Capacity per Trip (ci): RemainingCapacity + 40 to account for overcrowding scenarios.",
    "  - Current Trips (ti): Scheduled trips, applied uniformly to stations on a line and direction.",
    "  - Additional Trips (xi): Extra trips needed, averaged across days and rounded up.",
    "  - Weights (wi): Number of unique lines per station.",
    "- Optimization:",
    "  - Daily optimizations use adjusted capacity and demand.",
    "  - Unweighted: Minimize sum(xi).",
    "  - Weighted: Minimize sum(wi * xi).",
    "  - Constraints: ti * ci + xi * ci >= di.",
    "",
    "Results by Transfer Station",
    "--------------------------",
]
unweighted_table = tabulate(avg_results, headers='keys', tablefmt='grid', showindex=False)
weighted_table = tabulate(avg_weighted_results, headers='keys', tablefmt='grid', showindex=False)

report_lines.append("Unweighted Optimization (Average Additional Trips):")
report_lines.append(unweighted_table)
report_lines.append("")

report_lines.append("Weighted Optimization (Average Additional Trips):")
report_lines.append(weighted_table)
report_lines.append("")

results_by_station = defaultdict(list)
for _, result in avg_results.iterrows():
    results_by_station[result['Station']].append(result)
weighted_results_by_station = defaultdict(list)
for _, result in avg_weighted_results.iterrows():
    weighted_results_by_station[result['Station']].append(result)

for station in sorted(transfer_stations):
    report_lines.append(f"{station}")
    report_lines.append("-" * len(station))
    report_lines.append(f"Number of Lines: {weights[station]}")
    report_lines.append(f"Weight: {weights[station]} (based on number of lines)")
    
    station_results = results_by_station.get(station, [])
    if station_results:
        report_lines.append("Unweighted Optimization (Average Additional Trips):")
        for result in station_results:
            report_lines.append(
                f"- Line {result['Line']}, Direction {result['Direction']}, Hour {int(result['Hour'])}: "
                f"{int(result['Additional_Trips'])} trips needed, demand {result['Passenger_Demand']:.2f} passengers, "
                f"current trips: {result['Current_Trips']:.0f}, capacity: {result['Capacity_Per_Trip']:.2f} passengers."
            )
    else:
        report_lines.append("Unweighted Optimization: No additional trips needed.")
    
    station_weighted_results = weighted_results_by_station.get(station, [])
    if station_weighted_results:
        report_lines.append("Weighted Optimization (Average Additional Trips):")
        for result in station_weighted_results:
            report_lines.append(
                f"- Line {result['Line']}, Direction {result['Direction']}, Hour {int(result['Hour'])}: "
                f"{int(result['Additional_Trips'])} trips needed, demand {result['Passenger_Demand']:.2f} passengers, "
                f"current trips: {result['Current_Trips']:.0f}, capacity: {result['Capacity_Per_Trip']:.2f} passengers."
            )
    else:
        report_lines.append("Weighted Optimization: No additional trips needed.")
    
    report_lines.append("Insights:")
    if station_results or station_weighted_results:
        report_lines.append(
            f"- {station} experiences overcrowding, requiring additional trips. "
            f"The weighted optimization prioritizes it due to {weights[station]} line(s)."
        )
    else:
        report_lines.append(f"- {station} has sufficient capacity, suggesting no immediate need for additional services.")
    report_lines.append("")

total_avg_trips = avg_results['Additional_Trips'].sum()
total_weighted_avg_trips = avg_weighted_results['Additional_Trips'].sum()
report_lines.append("Summary")
report_lines.append("-------")
report_lines.append(f"Total Average Additional Trips (Unweighted): {total_avg_trips:.0f} trips.")
report_lines.append(f"Total Average Weighted Additional Trips: {total_weighted_avg_trips:.0f} trips.")
report_lines.append("Recommendations:")
report_lines.append("- Prioritize stations with higher weights for resource allocation.")
report_lines.append("- Monitor stations requiring additional trips for infrastructure upgrades.")
report_lines.append("- Consider operational constraints (e.g., maximum trips per hour).")
report_lines.append("- Verify adjusted capacity data to ensure realistic trip estimates.")

# Save plain text report
with open('../Data/output/reports/optimization_report.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(report_lines))

# Print summary
print(f"\nTotal Average Additional Trips (Unweighted): {total_avg_trips:.0f}")
print(f"Total Weighted Average Additional Trips: {total_weighted_avg_trips:.0f}")
print("\nDetailed report saved to 'optimization_report.txt'")
print("Daily results saved to 'optimization_results.csv' and 'weighted_optimization_results.csv'")
print("Averaged results saved to 'avg_optimization_results.csv' and 'avg_weighted_optimization_results.csv'")

Columns in metro_capacities_updated.csv: ['Line', 'Station', 'Direction', 'transition_date', 'transition_hour', 'number_of_passenger', 'RemainingCapacity']
Columns in metro_schedule.csv: ['Line', 'transition_hour', 'Number_of_Trips', 'Direction', 'Day_Type', 'Peak_Hour_Indicator']

Unique stations in capacities_df: ['Hacıosman' 'Darüşşafaka' 'Atatürk Oto Sanayi' 'Ayazağa-İTÜ' 'Seyrantepe'
 'Sanayi Mahallesi' '4.Levent' 'Levent' 'Gayrettepe' 'Şişli-Mecidiyeköy'
 'Osmanbey' 'Taksim' 'Şişhane' 'Haliç' 'Vezneciler-İstanbul Üniv.'
 'Yenikapı' 'Bakırköy Sahil' 'Özgürlük Meydanı' 'İncirli' 'Haznedar'
 'İlkyuva' 'Molla Gürani' 'Kirazlı-Bağcılar' 'Yenimahalle' 'Mahmutbey'
 'İSTOÇ' 'İkitelli Sanayi' 'Turgut Özal' 'Siteler' 'Başak Konutları'
 'Başakşehir-Metrokent' 'Onurkent' 'Şehir Hastanesi' 'Toplu Konutlar'
 'Kayaşehir Merkez' 'Kadıköy' 'Ayrılık Çeşme' 'Acıbadem' 'Ünalan'
 'Göztepe' 'Yenisahra' 'Kozyatağı' 'Bostancı' 'Küçükyalı' 'Maltepe'
 'Huzurevi' 'Gülsuyu' 'Esenkent' 'Hastane – Adliye' 'So